In [1]:
from pandas import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#define a function to extract patient_name from the excel
def patient_name(row):
    sample=row['Sample ID'].strip().split()
    for value in row['Sample ID'].strip().split():
        if value in ['v1','v2','v3','V1','V2','V3','10','100','1000','10000','100000','1000000','10000000']:
            sample.remove(value)
    return '_'.join(sample)


#define a function to return a name set for each sheet
def patient_name_set(df):
    set_name=set()
    for i, row in df.iterrows():
        set_name.add(row['patient_name'])
    return set_name

#define a function to extract the patient title
def plot_title(df):
    patients={}
    for i, row in df.iterrows():
        if isinstance(row['Hospital '], str):
            patients[row["patient_name"]] = "_".join([row['Hospital '], str(int(row['Age'])), 'year', row['Gender']])
    plot_title=patients
    return plot_title

In [2]:
#list of target moleculars for plots
colls=['PSMalpha2', 'PSMalpha3',
       'psmalpah4', 'BSA', 'Betatoxin', 'hIgA', 'LDL', 'SEB',
       'S.Pyogenese arcA', 'LukE', 'Pn PS12', 'LukD', 'Pn PS23', 'HLA-1',
       'SpA domD5-WT', 'Glom.extract', 'SpA domD5FcNull', 'SEN', 'hIgG', 'SEU',
       'HLA', 'SEI', 'LukAB(Lab)', 'SEM', 'LukAB(cc30)', 'surface protein ext',
       'SEB.1', 'cytoplasmic ext', 'Hemolysin gamma A', 'Pn CWPS',
       'Hemolysin gamma B', 'ABA', 'Hemolysin gamma C', 'PC-12', 'LukS-PV',
       'SEO', 'SP', 'SEG', 'PLY', 'HSA', 'Exoprotein ext', 'Rabbit IgG',
       'LukF-PV', 'PSM 4variant', 'PC4', 'PNAG', 'PC16', 'HLA -2',
       'Tetanus Toxoid']



In [3]:
dfs=[]
for i in range(11):
    #read all the sheets to a list of dataframes
    dfs.append(pandas.read_excel(open('06222016 Staph Array Data.xlsx','rb'), skiprows=1, sheetname=i))
    #extract dilution value from the excel  
    dfs[i]['dilution']=dfs[i].apply(lambda row: row['Sample ID'].strip().split()[-1] if row['Sample ID'].strip().split()[-1] in ['10','100','1000','10000','100000','1000000','10000000'] else 'NONE', axis=1)
    #extract visit times from the excel
    dfs[i]['visit_time']=dfs[i].apply(lambda row: row['Sample ID'].strip().split()[-2] if len(row['Sample ID'].strip().split())>1 and row['Sample ID'].strip().split()[-2] in ['v1','v2','v3','V1','V2','V3'] else 'NONE', axis=1)
    #extract dilution value from the excel
    dfs[i]['patient_name']=dfs[i].apply(patient_name,axis=1)
    #use the function to get the patien_name list
    patient_names=patient_name_set(dfs[i])
    
    
    for patient in patient_names:
        #filtered dataframe to one patient
        df_sp=dfs[i].loc[dfs[i]['patient_name'] == patient]
        #plotting for each patient for specific molecule
        for molecule in colls:
            #filtered dataframe to one patient, one molecule
            df_sp_sm=df_sp.ix[:,[molecule,'visit_time','dilution']]
            #transfer column-dilution to interger
            df_sp_sm["dilution1"] = df_sp_sm.apply(lambda row: int(row["dilution"]), axis=1)
            #ploting lines in one plot, one line for one visit
            fig, ax = plt.subplots(1,1)
            labels = []
            for key, grp in df_sp_sm.groupby(['visit_time']):
                ax.set_xscale('log')
                ax.set_yscale('log')
                ax = grp.plot(ax=ax, kind='line', x='dilution1', y= molecule, marker='o',title=plot_title(df_sp))
                labels.append(key)
            lines, _ = ax.get_legend_handles_labels()
            ax.legend(lines, labels, loc='best')

            plt.xlabel("Dilution")
            plt.ylabel("Intensity")
            plt.show()
            

SyntaxError: unexpected EOF while parsing (<ipython-input-3-1ffb2a1edd73>, line 9)